In [13]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Read Excel data into DataFrames
df_species = pd.read_excel('species_present_at_mgp.xlsx')
df_color_coded = pd.read_excel('invasive plant priorities and visibility months static copy.xlsx')

# Remove spaces from column names
df_species.columns = df_species.columns.str.replace(' ', '')
df_color_coded.columns = df_color_coded.columns.str.replace(' ', '')

# Merge tables based on species column
merged_df = pd.merge(df_species, df_color_coded, on='Species', how='left')


# Load existing workbook
wb = load_workbook('invasive plant priorities and visibility months static copy.xlsx')
ws = wb.active  # Assuming you're working with the active sheet

# Create a summary table
summary_table = merged_df.copy()  # Copy the merged DataFrame

# Specify the columns to include in the summary table
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Iterate over months and retrieve color-coded data
for month in months:
    # Find the column letter for the month in the original workbook
    for col in ws.iter_cols(min_col=1, max_col=ws.max_column, min_row=1, max_row=1):
        for cell in col:
            if month.lower() in cell.value.lower():
                col_letter = cell.column_letter
                # Copy the cell values and colors to the summary table
                for i, species in enumerate(df_species['Species']):
                    if species in merged_df['Species'].values:
                        val = merged_df.loc[merged_df['Species'] == species, month].iloc[0]
                        summary_table.at[i, month + '_color'] = val
                        summary_table.at[i, month + '_color_fill'] = cell.fill.start_color.rgb  # Store the color RGB value
                    else:
                        summary_table.at[i, month + '_color'] = 'Default Value'  # Filling missing data with a default value
                        summary_table.at[i, month + '_color_fill'] = 'Default Color'  # Filling missing color with a default color
                break  # Move to the next month
        if month.lower() in cell.value.lower():
            break  # Exit the loop if the month column is found
# Save the modified workbook
wb.save('summary_table.xlsx')

# Load existing workbook
wb = load_workbook('invasive plant priorities and visibility months static copy.xlsx')
ws = wb.active  # Assuming you're working with the active sheet

# Create separate DataFrames for species in specified months and all other species
filtered_species = pd.DataFrame(columns=merged_df.columns)  # Initialize DataFrame for species in specified months
other_species = pd.DataFrame(columns=merged_df.columns)  # Initialize DataFrame for all other species

# Specify the months
months = ['February', 'March', 'April']

# Function to get cell color
def get_cell_color(cell):
    if cell.fill.start_color.index == '00000000':
        return None  # Return None if the cell has no color
    return cell.fill.start_color.rgb  # Return the color value

# Iterate over rows and filter species
for index, row in merged_df.iterrows():
    month_colors = []
    for month in months:
        col_idx = None
        for col in ws.iter_cols(min_col=1, max_col=ws.max_column, min_row=1, max_row=1):
            for cell in col:
                if month.lower() in cell.value.lower():
                    col_idx = cell.column
                    break
            if col_idx:
                break
        if col_idx:
            cell_color = get_cell_color(ws.cell(row=index+2, column=col_idx))
            month_colors.append(cell_color)
    if 'FF00FF00' in month_colors or 'FFFFFF00' in month_colors:  # Green or Yellow color codes
        filtered_species = filtered_species.append(row, ignore_index=True)
    else:
        other_species = other_species.append(row, ignore_index=True)

# Save the filtered species and other species to separate Excel files with cell formatting
with pd.ExcelWriter('filtered_species.xlsx', engine='openpyxl') as writer:
    writer.book = wb  # Use the existing workbook
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)  # Retain existing sheets

    # Write filtered species with cell formatting
    filtered_species.to_excel(writer, sheet_name='Filtered Species', index=False)

with pd.ExcelWriter('other_species.xlsx', engine='openpyxl') as writer:
    writer.book = wb  # Use the existing workbook
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)  # Retain existing sheets

    # Write other species with cell formatting
    other_species.to_excel(writer, sheet_name='Other Species', index=False)


AttributeError: 'DataFrame' object has no attribute 'append'

In [41]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Define a mapping of fill colors to words
color_mapping = {
    '00000000': 'No Color',
    'FFFF0000': 'Red',
    'FF00B050': 'Green',
    'FFFFC000': 'Yellow'
}

# Load existing workbook
wb = load_workbook('data inputs\priority invasive plants and visibility months.xlsx')
ws = wb.active  # Assuming you're working with the active sheet

# Get the column headers (assuming the headers are in the first row)
column_headers = [cell.value.strip() if cell.value else None for cell in ws[1]]

# Months without leading or trailing spaces
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Iterate over cells in the sheet and replace fill colors with corresponding words for columns corresponding to months
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):  # Start from the second row to skip header
    for i, cell in enumerate(row):
        # Get the fill color of the cell
        fill_color = cell.fill.start_color.rgb
        # If the column header corresponds to a month, replace fill color with corresponding word if present in mapping
        if column_headers[i] and column_headers[i] in months:
            if fill_color in color_mapping:
                cell.value = color_mapping[fill_color]

# Save the modified workbook
wb.save('invasive_plant_priorities_modified.xlsx')

In [37]:
import pandas as pd

# Read the modified Excel file into a DataFrame
df_modified = pd.read_excel('invasive_plant_priorities_modified.xlsx')

# Define months of interest
months_of_interest = ['February', 'March', 'April']

# Filter species where color is yellow or green in February, March, or April
filtered_species = df_modified[
    ((df_modified['February'] == 'Yellow') | (df_modified['February'] == 'Green')) |
    ((df_modified['March'] == 'Yellow') | (df_modified['March'] == 'Green')) |
    ((df_modified['April'] == 'Yellow') | (df_modified['April'] == 'Green'))
]

# Create another table with all other species
other_species = df_modified[~df_modified.index.isin(filtered_species.index)]

# Save both DataFrames to different tabs in the same Excel file
with pd.ExcelWriter('.xlsx') as writer:
    filtered_species.to_excel(writer, sheet_name='Filtered_Species', index=False)
    other_species.to_excel(writer, sheet_name='Other_Species', index=False)